# 🧠 Encephlo — DenseNet121 Training Pipeline
**Brain Tumor MRI Classification | Grad-CAM Compatible | Target: ≥96% Accuracy**

Classes: `glioma` | `meningioma` | `notumor` | `pituitary`

Architecture strategy:
- **Phase 1** — Freeze DenseNet121 base, train new top layers (warm-up)
- **Phase 2** — Unfreeze last 50 layers, fine-tune at low LR

Grad-CAM target layer: `conv5_block16_concat` (last concat in Dense Block 4)

## 1. 📦 Imports & Seed

In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cv2
from PIL import Image

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (
    EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
)

from sklearn.metrics import (
    classification_report, confusion_matrix, ConfusionMatrixDisplay
)

# ── Reproducibility ──────────────────────────────────────────────────────────
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

print(f'TensorFlow version : {tf.__version__}')
print(f'GPU available      : {tf.config.list_physical_devices("GPU")}')

## 2. ⚙️ Configuration

In [ ]:
# ── Paths ────────────────────────────────────────────────────────────────────
BASE_DIR     = os.path.abspath(os.path.join(os.getcwd(), '..'))
TRAIN_DIR    = os.path.join(BASE_DIR, '..', 'MRI images', 'Training')
TEST_DIR     = os.path.join(BASE_DIR, '..', 'MRI images', 'Testing')
MODELS_DIR   = os.path.join(BASE_DIR, 'models')
os.makedirs(MODELS_DIR, exist_ok=True)

MODEL_SAVE_PATH = os.path.join(MODELS_DIR, 'densenet121.keras')
BEST_WEIGHTS    = os.path.join(MODELS_DIR, 'densenet121_best.weights.h5')

# ── Hyperparameters ──────────────────────────────────────────────────────────
IMG_SIZE     = (224, 224)
IMG_SHAPE    = (224, 224, 3)
BATCH_SIZE   = 32
VAL_SPLIT    = 0.15            # 15% of training set for validation
CLASS_NAMES  = ['glioma', 'meningioma', 'notumor', 'pituitary']
NUM_CLASSES  = len(CLASS_NAMES)

# Phase 1 — warm-up (frozen base)
P1_EPOCHS    = 15
P1_LR        = 1e-3

# Phase 2 — fine-tune
P2_EPOCHS    = 35
P2_LR        = 1e-5
UNFREEZE_FROM = -50           # unfreeze last 50 layers for fine-tuning

# Grad-CAM target layer name (last conv concat in DenseNet121 Dense Block 4)
GRADCAM_LAYER = 'conv5_block16_concat'

print(f'Train dir  : {TRAIN_DIR}')
print(f'Test  dir  : {TEST_DIR}')
print(f'Models dir : {MODELS_DIR}')

# Quick sanity: count images per class
print('\n── Training class distribution ──')
for cls in CLASS_NAMES:
    path = os.path.join(TRAIN_DIR, cls)
    count = len(os.listdir(path)) if os.path.exists(path) else 0
    print(f'  {cls:<15}: {count} images')

## 3. 📂 Data Pipeline

In [ ]:
# ── Augmentation for training ─────────────────────────────────────────────────
train_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_input,   # DenseNet121-specific normalisation
    validation_split       = VAL_SPLIT,
    rotation_range         = 20,
    width_shift_range      = 0.15,
    height_shift_range     = 0.15,
    shear_range            = 0.1,
    zoom_range             = 0.2,
    horizontal_flip        = True,
    brightness_range       = [0.8, 1.2],
    fill_mode              = 'nearest'
)

# ── No augmentation for test/val ─────────────────────────────────────────────
val_test_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_input
)

# ── Generators ───────────────────────────────────────────────────────────────
train_gen = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size   = IMG_SIZE,
    batch_size    = BATCH_SIZE,
    class_mode    = 'categorical',
    classes       = CLASS_NAMES,
    subset        = 'training',
    seed          = SEED,
    shuffle       = True
)

val_gen = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size   = IMG_SIZE,
    batch_size    = BATCH_SIZE,
    class_mode    = 'categorical',
    classes       = CLASS_NAMES,
    subset        = 'validation',
    seed          = SEED,
    shuffle       = False
)

test_gen = val_test_datagen.flow_from_directory(
    TEST_DIR,
    target_size   = IMG_SIZE,
    batch_size    = BATCH_SIZE,
    class_mode    = 'categorical',
    classes       = CLASS_NAMES,
    shuffle       = False
)

print(f'\nTrain batches : {len(train_gen)}')
print(f'Val   batches : {len(val_gen)}')
print(f'Test  batches : {len(test_gen)}')
print(f'Class indices : {train_gen.class_indices}')

In [ ]:
# ── Visualise a batch sample ─────────────────────────────────────────────────
fig, axes = plt.subplots(2, 4, figsize=(14, 7))
batch_imgs, batch_labels = next(train_gen)

for i, ax in enumerate(axes.flat):
    # Denormalise DenseNet preprocess for display
    img_display = batch_imgs[i].copy()
    img_display = (img_display - img_display.min()) / (img_display.max() - img_display.min())
    ax.imshow(img_display)
    label_idx = np.argmax(batch_labels[i])
    ax.set_title(CLASS_NAMES[label_idx], fontsize=10)
    ax.axis('off')

plt.suptitle('Sample Augmented Training Images', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

## 4. 🏗️ Model Architecture (Grad-CAM Compatible Flat Model)

In [ ]:
def build_densenet121_model(num_classes: int, input_shape: tuple) -> Model:
    """
    Builds a FLAT (non-nested) DenseNet121 classifier.
    
    CRITICAL for Grad-CAM: We DO NOT wrap the base in a separate Model().
    Instead, we call base_model.layers directly into a functional API chain.
    This ensures make_gradcam_heatmap() in utils.py works with the
    'STANDARD FLAT MODEL' branch (line 41 of utils.py).
    
    Grad-CAM target layer: 'conv5_block16_concat'
    This is the output of the last dense block in DenseNet121 — rich
    spatial features, perfect for visualising tumour localisation.
    """
    # Load pre-trained base (ImageNet weights)
    base = DenseNet121(
        include_top  = False,
        weights      = 'imagenet',
        input_shape  = input_shape
    )
    base.trainable = False   # Freeze for warm-up phase
    
    # ── Functional API — FLAT chain (Grad-CAM requirement) ───────────────────
    inputs  = keras.Input(shape=input_shape, name='input_layer')
    x       = base(inputs, training=False)          # Base outputs feature maps
    
    # NOTE: because base is called as a callable (not wrapped in Model()),
    # the resulting graph is FLAT — utils.py's standard branch handles this.
    # However, since base IS a sub-model object, utils.py will detect it via
    # isinstance(layer, tf.keras.Model). We handle this by INSTEAD using
    # the Functional API properly so the inner_model branch works correctly.
    
    x       = layers.GlobalAveragePooling2D(name='gap')(x)
    x       = layers.BatchNormalization(name='bn_top')(x)
    x       = layers.Dense(512, activation='relu', name='fc_512')(
                  x, )
    x       = layers.Dropout(0.4, name='dropout_1')(x)
    x       = layers.Dense(256, activation='relu', name='fc_256')(x)
    x       = layers.Dropout(0.3, name='dropout_2')(x)
    outputs = layers.Dense(num_classes, activation='softmax', name='predictions')(x)
    
    model   = Model(inputs=inputs, outputs=outputs, name='Encephlo_DenseNet121')
    return model, base


model, base_model = build_densenet121_model(NUM_CLASSES, IMG_SHAPE)
model.summary()

print(f'\nTotal layers in base  : {len(base_model.layers)}')
print(f'Trainable params (P1) : {sum([p.numpy().size for p in model.trainable_weights]):,}')

In [ ]:
# ── Verify Grad-CAM target layer exists ──────────────────────────────────────
try:
    target_layer = base_model.get_layer(GRADCAM_LAYER)
    print(f'✅ Grad-CAM layer found: "{GRADCAM_LAYER}"')
    print(f'   Output shape: {target_layer.output_shape}')
except ValueError as e:
    print(f'❌ Layer not found: {e}')
    print('Available last-block layers:')
    for l in base_model.layers:
        if 'block16' in l.name or 'block15' in l.name:
            print(f'  {l.name}')

## 5. ⚡ Phase 1 — Warm-Up Training (Frozen Base)

In [ ]:
# ── Callbacks ────────────────────────────────────────────────────────────────
callbacks_p1 = [
    EarlyStopping(
        monitor              = 'val_accuracy',
        patience             = 5,
        restore_best_weights = True,
        verbose              = 1
    ),
    ReduceLROnPlateau(
        monitor  = 'val_loss',
        factor   = 0.5,
        patience = 3,
        min_lr   = 1e-7,
        verbose  = 1
    )
]

# ── Compile ───────────────────────────────────────────────────────────────────
model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=P1_LR),
    loss      = 'categorical_crossentropy',
    metrics   = ['accuracy']
)

print('── Phase 1: Warm-up (base frozen) ──────────────────────────────────────')
print(f'Learning rate : {P1_LR}')
print(f'Max epochs    : {P1_EPOCHS}')
print(f'Trainable params: {model.trainable_variables[-1].shape}')
print('Launching training...')

In [ ]:
history_p1 = model.fit(
    train_gen,
    validation_data  = val_gen,
    epochs           = P1_EPOCHS,
    callbacks        = callbacks_p1,
    verbose          = 1
)

p1_best_val_acc = max(history_p1.history['val_accuracy'])
print(f'\n── Phase 1 Complete ──────────────────────────────────────────────────')
print(f'Best val accuracy: {p1_best_val_acc:.4f} ({p1_best_val_acc*100:.2f}%)')

In [ ]:
# ── Phase 1 Learning Curves ─────────────────────────────────────────────────
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(history_p1.history['accuracy'],     label='Train Acc',  color='#2196F3')
ax1.plot(history_p1.history['val_accuracy'], label='Val Acc',    color='#4CAF50')
ax1.set_title('Phase 1 — Accuracy', fontweight='bold')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()
ax1.grid(alpha=0.3)

ax2.plot(history_p1.history['loss'],     label='Train Loss', color='#F44336')
ax2.plot(history_p1.history['val_loss'], label='Val Loss',   color='#FF9800')
ax2.set_title('Phase 1 — Loss', fontweight='bold')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()
ax2.grid(alpha=0.3)

plt.suptitle('DenseNet121 — Phase 1 Training (Frozen Base)', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

## 6. 🔓 Phase 2 — Fine-Tuning (Unfreeze Last 50 Layers)

In [ ]:
# ── Unfreeze last N layers of the base model ─────────────────────────────────
base_model.trainable = True

# Freeze all layers EXCEPT the last UNFREEZE_FROM (-50 means last 50)
for layer in base_model.layers[:UNFREEZE_FROM]:
    layer.trainable = False

trainable_count = sum(1 for l in base_model.layers if l.trainable)
print(f'Trainable base layers : {trainable_count} / {len(base_model.layers)}')

# ── Callbacks for Phase 2 ────────────────────────────────────────────────────
callbacks_p2 = [
    EarlyStopping(
        monitor              = 'val_accuracy',
        patience             = 10,
        restore_best_weights = True,
        verbose              = 1
    ),
    ReduceLROnPlateau(
        monitor  = 'val_loss',
        factor   = 0.3,
        patience = 4,
        min_lr   = 1e-8,
        verbose  = 1
    ),
    ModelCheckpoint(
        filepath             = BEST_WEIGHTS,
        monitor              = 'val_accuracy',
        save_best_only       = True,
        save_weights_only    = True,
        verbose              = 1
    )
]

# ── Recompile at VERY low LR ──────────────────────────────────────────────────
model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=P2_LR),
    loss      = 'categorical_crossentropy',
    metrics   = ['accuracy']
)

print(f'\n── Phase 2: Fine-Tuning ─────────────────────────────────────────────')
print(f'Learning rate : {P2_LR}')
print(f'Max epochs    : {P2_EPOCHS}')
print('Launching fine-tuning...')

In [ ]:
history_p2 = model.fit(
    train_gen,
    validation_data  = val_gen,
    epochs           = P2_EPOCHS,
    callbacks        = callbacks_p2,
    verbose          = 1
)

p2_best_val_acc = max(history_p2.history['val_accuracy'])
print(f'\n── Phase 2 Complete ──────────────────────────────────────────────────')
print(f'Best val accuracy: {p2_best_val_acc:.4f} ({p2_best_val_acc*100:.2f}%)')

# Best weights are already restored by ModelCheckpoint + EarlyStopping
model.load_weights(BEST_WEIGHTS)

In [ ]:
# ── Combined Training History ────────────────────────────────────────────────
def combine_histories(h1, h2):
    combined = {}
    for key in h1.history:
        combined[key] = h1.history[key] + h2.history[key]
    return combined

full_history = combine_histories(history_p1, history_p2)
p1_end = len(history_p1.history['accuracy'])

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

for ax, metric, ylabel in [
    (ax1, 'accuracy', 'Accuracy'),
    (ax2, 'loss',     'Loss')
]:
    ax.plot(full_history[metric],         label=f'Train {ylabel}', color='#2196F3')
    ax.plot(full_history[f'val_{metric}'],label=f'Val {ylabel}',   color='#4CAF50')
    ax.axvline(x=p1_end - 0.5, color='gray', linestyle='--', alpha=0.7, label='Phase 1 → Phase 2')
    ax.set_title(f'Full Training — {ylabel}', fontweight='bold')
    ax.set_xlabel('Epoch')
    ax.set_ylabel(ylabel)
    ax.legend()
    ax.grid(alpha=0.3)

plt.suptitle('DenseNet121 — Full Training History', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

## 7. 📊 Evaluation on Test Set

In [ ]:
# ── Test Accuracy & Loss ─────────────────────────────────────────────────────
print('Evaluating on unseen test set...')
test_loss, test_acc = model.evaluate(test_gen, verbose=1)

print(f'\n══════════════════════════════════════════')
print(f'  TEST ACCURACY  : {test_acc*100:.2f}%')
print(f'  TEST LOSS      : {test_loss:.4f}')
print(f'══════════════════════════════════════════')

if test_acc >= 0.96:
    print(f'  ✅ TARGET MET — {test_acc*100:.2f}% ≥ 96.00%')
else:
    print(f'  ⚠️  Below target — {test_acc*100:.2f}% < 96.00%')
    print('     Consider running more fine-tune epochs or adjusting LR.')

In [ ]:
# ── Predictions for Classification Report + Confusion Matrix ─────────────────
test_gen.reset()
y_pred_probs = model.predict(test_gen, verbose=1)
y_pred       = np.argmax(y_pred_probs, axis=1)
y_true       = test_gen.classes

print('\n── Classification Report ────────────────────────────────────────────')
print(classification_report(y_true, y_pred, target_names=CLASS_NAMES, digits=4))

In [ ]:
# ── Confusion Matrix ─────────────────────────────────────────────────────────
cm = confusion_matrix(y_true, y_pred)

fig, ax = plt.subplots(figsize=(8, 7))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=CLASS_NAMES)
disp.plot(
    ax          = ax,
    cmap        = 'Blues',
    colorbar    = False,
    values_format = 'd'
)
ax.set_title('DenseNet121 — Confusion Matrix (Test Set)', fontsize=13, fontweight='bold')
plt.xticks(rotation=30, ha='right')
plt.tight_layout()
plt.show()

# Per-class accuracy
print('\n── Per-Class Accuracy ───────────────────────────────────────────────')
for i, cls in enumerate(CLASS_NAMES):
    cls_acc = cm[i, i] / cm[i].sum()
    print(f'  {cls:<15}: {cls_acc*100:.2f}%')

## 8. 💾 Save Model

In [ ]:
model.save(MODEL_SAVE_PATH)
print(f'\n✅ Model saved to: {MODEL_SAVE_PATH}')
print(f'   File size: {os.path.getsize(MODEL_SAVE_PATH) / 1e6:.1f} MB')

## 9. 🔥 Grad-CAM Smoke Test
Verifies the model is fully compatible with `utils.py`'s `make_gradcam_heatmap()` function.

In [ ]:
# ── Inline Grad-CAM (mirrors utils.py logic exactly) ─────────────────────────
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    """
    Exact copy of Encephlo's utils.py — validates compatibility.
    The model has a nested base (DenseNet121), so the inner_model branch runs.
    """
    inner_model = next(
        (layer for layer in model.layers if isinstance(layer, tf.keras.Model)), None
    )

    if inner_model is not None:
        last_conv_layer = inner_model.get_layer(last_conv_layer_name)
        grad_base_model = tf.keras.Model(
            inner_model.inputs,
            [last_conv_layer.output, inner_model.output]
        )

        with tf.GradientTape() as tape:
            conv_outputs, base_outputs = grad_base_model(img_array)
            tape.watch(conv_outputs)
            x = base_outputs
            inner_idx = model.layers.index(inner_model)
            for layer in model.layers[inner_idx + 1:]:
                x = layer(x)
            preds = x
            if pred_index is None:
                pred_index = tf.argmax(preds[0])
            class_channel = preds[:, pred_index]

        grads = tape.gradient(class_channel, conv_outputs)
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
        last_conv_layer_output = conv_outputs[0]
        heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
        heatmap = tf.squeeze(heatmap)
        heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
        return heatmap.numpy()
    else:
        grad_model = tf.keras.Model(
            inputs=model.inputs,
            outputs=[model.get_layer(last_conv_layer_name).output, model.output]
        )
        with tf.GradientTape() as tape:
            outputs = grad_model(img_array)
            last_conv_layer_output, preds = outputs
            if pred_index is None:
                pred_index = tf.argmax(preds[0])
            class_channel = preds[:, pred_index]
        grads = tape.gradient(class_channel, last_conv_layer_output)
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
        last_conv_layer_output = last_conv_layer_output[0]
        heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
        heatmap = tf.squeeze(heatmap)
        heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
        return heatmap.numpy()


def overlay_heatmap(img, heatmap, alpha=0.4):
    h, w = img.shape[:2] if hasattr(img, 'shape') else (img.size[1], img.size[0])
    heatmap_uint8 = np.uint8(255 * heatmap)
    heatmap_resized = cv2.resize(heatmap_uint8, (w, h))
    heatmap_bgr = cv2.applyColorMap(heatmap_resized, cv2.COLORMAP_JET)
    heatmap_rgb = cv2.cvtColor(heatmap_bgr, cv2.COLOR_BGR2RGB)
    img_array = np.array(img)
    superimposed = heatmap_rgb * alpha + img_array
    return np.clip(superimposed, 0, 255).astype('uint8')


print('Grad-CAM functions loaded ✅')

In [ ]:
# ── Pick one sample from each class and run Grad-CAM ─────────────────────────
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
sample_errors = []

for col, cls in enumerate(CLASS_NAMES):
    cls_dir = os.path.join(TEST_DIR, cls)
    sample_file = os.path.join(cls_dir, os.listdir(cls_dir)[0])
    
    raw_img = Image.open(sample_file).convert('RGB').resize(IMG_SIZE)
    img_arr = np.array(raw_img)
    img_prep = preprocess_input(np.expand_dims(img_arr.copy(), axis=0))
    
    try:
        # ── Run prediction ────────────────────────────────────────────────────
        preds     = model.predict(img_prep, verbose=0)
        pred_cls  = CLASS_NAMES[np.argmax(preds)]
        pred_conf = preds[0][np.argmax(preds)] * 100
        
        # ── Generate Grad-CAM heatmap ─────────────────────────────────────────
        heatmap   = make_gradcam_heatmap(img_prep, model, GRADCAM_LAYER)
        overlay   = overlay_heatmap(raw_img, heatmap)
        
        # ── Display ───────────────────────────────────────────────────────────
        axes[0][col].imshow(raw_img)
        axes[0][col].set_title(f'Original\n[{cls}]', fontsize=9)
        axes[0][col].axis('off')
        
        axes[1][col].imshow(overlay)
        ok = '✅' if pred_cls == cls else '❌'
        axes[1][col].set_title(
            f'Grad-CAM {ok}\nPred: {pred_cls}\n{pred_conf:.1f}%', fontsize=8
        )
        axes[1][col].axis('off')
        
    except Exception as e:
        sample_errors.append(f'{cls}: {e}')
        axes[0][col].text(0.5, 0.5, f'Error\n{e}', ha='center', va='center', transform=axes[0][col].transAxes)
        axes[1][col].axis('off')

plt.suptitle(
    f'DenseNet121 — Grad-CAM Smoke Test\nLayer: "{GRADCAM_LAYER}"',
    fontsize=13, fontweight='bold'
)
plt.tight_layout()
plt.show()

if sample_errors:
    print('❌ Errors encountered:')
    for err in sample_errors:
        print(f'  {err}')
else:
    print('✅ Grad-CAM smoke test PASSED — fully compatible with utils.py')

## 10. ✅ Final Summary

In [ ]:
print('═' * 55)
print('  ENCEPHLO — DenseNet121 Training Summary')
print('═' * 55)
print(f'  Model         : DenseNet121 (ImageNet → Fine-tuned)')
print(f'  Classes       : {CLASS_NAMES}')
print(f'  Input size    : {IMG_SIZE}')
print(f'  Grad-CAM layer: {GRADCAM_LAYER}')
print(f'  Test Accuracy : {test_acc*100:.2f}%  (target: 96%+)')
print(f'  Test Loss     : {test_loss:.4f}')
print(f'  Model saved   : {MODEL_SAVE_PATH}')
print('═' * 55)
print()
print('ENSEMBLE_CONFIG entry for app.py:')
print('  "DenseNet121": {')
print(f'      "path" : "models/densenet121.keras",')
print(f'      "layer": "{GRADCAM_LAYER}",')
print( '      "preprocess": tf.keras.applications.densenet.preprocess_input')
print('  }')